In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

In [4]:
import tensorflow as tf
import json
import numpy as np
import random
import pandas as pd

In [5]:
from google.colab import files
files.upload()

Saving intents (1).json to intents (1) (1).json


{'intents (1) (1).json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\

In [6]:
with open('intents (1).json') as json_data:
  intents=json.load(json_data)

In [7]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [15]:
words=[]
classes=[]
documents=[]
ignore=["?","!"]

for intent in intents['intents']:
  for pattern in intent['patterns']:

    # tokenize each and every word
    w=nltk.word_tokenize(pattern)

    # add all the words in the word list
    words.extend(w)
    #Append function in Python adds a single element to the end of the list,
    #whereas the extend function adds multiple elements (from an iterable) to the list.

    #add words to documents
    documents.append((w,intent['tag']))
    # add tags to our classes list that is present in the json file
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

  # Perform stemming and lower each word as well as remove
  words = [stemmer. stem (w.lower()) for w in words if w not in ignore]
  words = sorted (list (set(words)))

# remove duplicate words
classes = sorted(list(set(classes)))

# removing duplicate classes

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

# stmmer does is convert any verb to its root form




27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodbi', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [16]:
# create training data
training = []         # train x
output = []           # y output

# create an empty array for output
output_empty = [0]* len(classes)
# create training set, bag of words for each sentence

#The bag-of-words model is a model of text which uses a representation of text
# that is based on an unordered collection (or "bag") of words.

for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer. stem (word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is '1' for current tag and '0' for rest of other tags
    output_row= list (output_empty)
    output_row[classes.index(doc [1])] = 1

    training.append([bag, output_row])
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)
# creating training lists
train_x= list(training[:,0])
train_y = list(training[:,1])

<ipython-input-16-558db44c666f>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


#now we are applying the model keras (we can use functional function but for begineers sequential is better)


In [19]:
model = tf. keras.Sequential()
model.add(tf.keras.layers.Dense(50,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense (10))
model.add(tf.keras.layers.Dense(len (train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)
model.save("model.pk1")

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 2.1148 - accuracy: 0.1852
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0492 - accuracy: 0.1852
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9914 - accuracy: 0.2593
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 1.9354 - accuracy: 0.2593
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8854 - accuracy: 0.3333
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.8336 - accuracy: 0.4074
Epoch 7/100
4/4 [==============================] - 0s 6ms/step - loss: 1.7823 - accuracy: 0.4815
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7356 - accuracy: 0.5185
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.6857 - accuracy: 0.5556
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.6398 - accuracy: 0.5556
Epoch 11/100
4/4 [===========

In [21]:
import pickle

In [27]:
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [ ]:
# if anytime there is a runtime issue we can start the program from here again
#by including the below statement

#from keras.models import load_model
#model = load_model("model.pkl")

In [26]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [29]:
with open('intents (1).json') as json_data:
    intents = json.load(json_data)

In [30]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [31]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

# Now aur chatbot is ready to answer (we have created less classes we can add them if we want to make it more complex chatbot)

# create intent file as per your requirements

In [39]:
response("where are u located")

1/1 [==============================] - 0s 20ms/step
We are from World's largest Democracy India.
